In [1]:
# IMPORT STATEMENTS
import pandas as pd

In [173]:
# SAVE KAPRSAC JET FUEL PRICES IN A DATAFRAME
df_org = pd.read_csv("KARPSAC-jetfuelprices-2010to2020.csv", sep=";") # original data set
df = df_org.copy() 

In [99]:
# PRELIMINARY STATS:
df.describe()

,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB in U.S. Dollars per Gallon
count,7626.000000
mean,1.407106
std,0.897850
min,0.282000
25%,0.581000
50%,1.199000
75%,2.007750
max,4.814000


In [174]:
# CLEAN UP DATA TO PERFORM GROUPBY ANALYSIS
df['YEAR'] = df['Date'].astype(str).str[0:4]
df = df[df['YEAR'].astype(int) >= 2010] # filters only years from 2010 to 2020
df['MONTH'] = df['Date'].astype(str).str[5:7]
df['DAY'] = df['Date'].astype(str).str[8:10]
del df['Date']

In [175]:
#CHANGE NUMERICAL MONTH TO MONTHS
month_lookup = {'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May',
                '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October', '11': 'November',
                '12': 'December'}

In [176]:
df['MONTH'] = df['MONTH'].apply(lambda x: month_lookup[x])

In [177]:
df['MONTH YEAR'] = df['MONTH'] + ' ' +  df['YEAR']
df.set_index('MONTH YEAR')

,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB in U.S. Dollars per Gallon,YEAR,MONTH,DAY
MONTH YEAR,,,,
August 2012,3.123,2012,August,07
August 2012,3.051,2012,August,05
August 2012,2.939,2012,August,01
July 2012,2.914,2012,July,30
July 2012,2.934,2012,July,15
...,...,...,...,...
September 2012,3.218,2012,September,05
August 2012,3.225,2012,August,26
August 2012,3.162,2012,August,19


In [185]:
# FIND MEAN OF THESE PRICES
df.groupby(['YEAR', 'MONTH']).mean()

U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB in U.S. Dollars per Gallon
YEAR MONTH                                                                                     
2010 April                                              2.242900                               
     August                                             2.081957                               
     December                                           2.456952                               
     February                                           2.001056                               
     January                                            2.047500                               
...                                                          ...                               
2020 January                                            1.769727                               
     July                                               1.056000                               
     June                                               0.989818                               
     March                                              0.940130                               
     May                                                0.704650                               

[127 rows x 1 columns]

In [183]:
df

,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB in U.S. Dollars per Gallon,YEAR,MONTH,DAY,MONTH YEAR
381,3.123,2012,August,07,August 2012
382,3.051,2012,August,05,August 2012
383,2.939,2012,August,01,August 2012
384,2.914,2012,July,30,July 2012
385,2.934,2012,July,15,July 2012
...,...,...,...,...,...
7621,3.218,2012,September,05,September 2012
7622,3.225,2012,August,26,August 2012
7623,3.162,2012,August,19,August 2012
7624,3.163,2012,August,16,August 2012
